### Series of ledger operations:

#### 0: create basic ledger from filenames

#### 1: Encode Am tags from ledger into filenames; only include expl and/or enc tagged

- name format:
    - vol_chap-date-nat-enc-expl-title-pagerange
    - enc_full enc_part enc_min enc_none
    - expl_y expl_n

underscores within features, dashes between features

- for entry in ledger:
    - if enc OR expl:
        - locate file by vol-chap
        - save file in new dir with new name

#### 2: prepare ledger for re-integration with rest of corpus
#### 3: update merged ledger with EN-only wordcounts

In [2]:
import csv
import os
import re
import string
punctuation = [i for i in string.punctuation]

In [7]:
def HK_date(fileid):
    #extracts date component from Hakluyt text file name, returns as integer
    return(int(fileid[11:15]))
def HK_geog(fileid):
    #extracts geography component from Hakluyt text file name, returns as string
    return(fileid[6:10])
def HK_title(fileid):
    #extracts geography component from Hakluyt text file name, returns as string
    return(fileid[16:-15])
def HK_vol(fileid):
    #extracts volume component from Hakluyt text file name, returns as integer
    return(int(fileid[:2]))
def HK_chap(fileid):
    #extracts chapter component from Hakluyt text file name, returns as integer
    return(int(fileid[3:5]))
def HK_pages(fileid):
    #extracts page range component from Hakluyt text file name, returns as tuple of integers
    pages_char = fileid[-11:-4]
    pages_char_split = pages_char.split('-')
    return(int(pages_char_split[0]), int(pages_char_split[1]))
def HK_page_length(fileid):
    #extracts page length component from Hakluyt text file name, returns as integer
    first_page, last_page = HK_pages(fileid)
    return(last_page-first_page+1)

In [9]:
with open('text-data/ledger.csv', 'w', newline='') as ledgerfile: 
    ledger = csv.writer(ledgerfile) 
    ledger.writerow(['vol', 'chap', 'geog', 'date', 'title', 'pages'])
    filelist = os.scandir('text-data/CambridgeCore MacLehose pdfminer extract')
    for file in filelist:
        ledger.writerow([HK_vol(file.name), HK_chap(file.name), HK_geog(file.name), HK_date(file.name), HK_title(file.name), str(HK_pages(file.name)[0]).zfill(3)+'-'+str(HK_pages(file.name)[1]).zfill(3)])

In [37]:
def get_vol(name):
    return(name[:2])
def get_chap(name):
    name = name[3:]
    return(name[:name.find('_')])
def wordcount(text):
    '''splits texts on whitespace and counts the segments, ignoring punctuation'''
    tokens = text.split()
    words = [w for w in tokens if w not in punctuation]
    return(len(words))
def wordcount_f(vol, chap, path):
    '''locates file according to vol/chap in path, splits texts on whitespace and counts the segments, ignoring punctuation'''
    filelist = os.scandir(path)
    for entry in filelist:
        if int(entry.name[:2]) == int(vol):
            #for full chapter files:
            if '.' not in chap:
                if int(entry.name[3:5]) == int(chap):
                    with open(entry.path, mode = 'r', encoding="utf8") as f:
                        text = f.read()
                        return(wordcount(text))
            else:
                #for American sectioned files:
                if entry.name[entry.name.find('_')+1:entry.name.find('AMER')-1] == chap:
                    with open(entry.path, mode = 'r', encoding="utf8") as f:
                        text = f.read()
                        return(wordcount(text))

In [24]:
# testseg = '07_17.2_AMER_1170_The_most_ancient_voyage_and_discovery_of_the_West_Indies_performed_by_Madoc_the_sonne_of_Owen_Guined_pp.133-135.txt'
# testfull = '09_24_AMER_1572_The_memorable_voyage_of_M_John_Chilton_to_all_the_principall_parts_of_Nueva_Espanna_pp.360-377'
# #print(get_chap(test))
# print(testfull[3:5])

24


In [25]:
# chap = '22.4'
# print(chap[:chap.find('.')])

22


In [26]:
# #spot-wordcounts for missing lines
# missing_files = ['11_29_AMER_1579_A_discourse_of_the_West_Indies_and_the_South_sea_written_by_Lopez_Vaz_a_Portugall_conteining_divers__pp.227-290.txt']
# missing_paths = ['text-data/CC_ML_FR_trimmed_morphad_lem_AMER/' + f for f in missing_files]
# for path in missing_paths:
#     with open(path, mode = 'r', encoding="utf8") as f:
#         text = f.read()
#         print(wordcount(text))

24102


#### 2: create an integrated full + AM_cat ledger

1. compute & record segment wordcount
1. [manually merge enc ledger with full ledger]
1. compute & record full chapter wordcounts
1. compute section fractions
1. update the following fields from sections into parent chapters: national_perspective, exploration_component, intercultural_encounter_component, minor_intercultural_encounter_component

In [29]:
# #now obsolete code merging AMledger with full ledger

# cat_path = 'text-data/CC_ML_FR_trimmed_MAlem_AMER_cat_EN/'
# lem_path = 'CC_ML_FR_trimmed_cleaned_EN_cl_MAlem'

# with open('text-data/ledger_merged_4.csv', mode = 'r') as ledger_csv:
#     full_ledger = csv.DictReader(ledger_csv) 
#     with open('text-data/AMledger_wordcounts.csv', mode = 'r') as AMledger_csv:
#         AM_ledger = csv.DictReader(AMledger_csv)
#         #writing header row
#         with open('text-data/ledger_merged_EN.csv', mode = 'a', newline='') as ledger_new_csv:
#             writer = csv.DictWriter(ledger_new_csv, ledger.fieldnames)
#             writer.writer.writerow(ledger.fieldnames)
#             #copy over all non-Am rows from full ledger + wordcounts + sect_f = 1 (includes header #2)
#             for row in full_ledger:
#                 if row['geog'] != 'AMER':
#                     #except header # 2 from wordcount/sect
#                     if row['vol'] != 'volume':
#                         row['word'] = wordcount_f(row['vol'], row['chap'])
#                         row['sect_frac'] = 1
#                     writer.writerow(row)
#             #for Am rows:
#             #for full chapters in am ledger, just copy am rows into the new ledgers  + sect_f = 1
#             #when catches section: find corresponding full chapter;  + wordcount + sect_f = 1
#             # then write in all the sectioned rows while calculating fractions
#             #starting scan on am_ledger
#             for row in AM_ledger:
#                 #for non-header rows:
#                 if row['geog'] == 'AMER':
#                     if '.' not in row['chap']:
#                         #for full chapters in am ledger, just copy am rows into the new ledgers +  sect_f = 1
#                         filelist = os.scandir(parent_path)
#                         row['sect_frac'] = 1
#                         writer.writerow(row)
#                     else:
#                         #sequence of sections begins:  find corresponding full chapter row, write in + wordcount + sect_f = 1
#                                                     # then write in all the sectioned rows while calculating fractions
#                         #print(row)
#                         if row['chap'][-1] == '1':
#                             #bring over row from ful ledger
#                             vol = row['vol']
#                             chap = row['chap'][:row['chap'].find('.')]
#                             print(f'looking for {vol}, {chap}')
#                             with open('text-data/ledger_merged_1.csv', mode = 'r') as ledger3_csv:
#                                 full_ledger2 = csv.DictReader(ledger3_csv) 
#                                 for row_f in full_ledger2:
#                                     #print(row_f)
#                                     if row_f['vol'] != 'volume':
#                                         if int(row_f['vol']) == int(vol) and int(row_f['chap']) == int(chap):
#                                             wordcount_full = wordcount_f(vol, chap)
#                                             row_f['word'] = wordcount_full
#                                             row_f['sect_frac'] = 1
#                                             writer.writerow(row_f)
#                                             break
#                         row['sect_frac'] = int(row['word']) / wordcount_full
#                         writer.writerow(row)

In [ ]:
# #checking Eric's suggestion of 17 mil words
# t_count = w_count = 0
# filelist = os.scandir('text-data/CC_ML_FR_trimmed_cleaned_MAspel')
# for entry in filelist:
#     with open(entry.path, mode = 'r', encoding="utf8") as f:
#         text = f.read()
#         tokens = text.split()
#         words = [w for w in tokens if w not in punctuation]
#         t_count += len(tokens)
#         w_count += len(words)
# print(f'{t_count} tokens, {w_count} words')

#### 3: update merged ledger with EN-only wordcounts

In [59]:
# #update merged ledger with EN-only wordcounts
# cat_path = 'text-data/CC_ML_FR_trimmed_MAlem_AMER_cat_EN/'
# lem_path = 'CC_ML_FR_trimmed_cleaned_EN_cl_MAlem'

# with open('text-data/ledger_merged_3.csv', mode = 'r') as ledger_csv:
#     full_ledger = csv.DictReader(ledger_csv) 
#     with open('text-data/ledger_merged_EN.csv', mode = 'a', newline='') as ledger_new_csv:
#         writer = csv.DictWriter(ledger_new_csv, full_ledger.fieldnames)
#         #writing header row
#         writer.writeheader()
#         #writer.writer.writerow(ledger.fieldnames)
#         #simply update non-Am rows from full ledger with new wordcounts
#         for row in full_ledger:
#             #add wordcount to row
#             if row['geog'] != 'AMER':
#                 row['word'] = wordcount_f(row['vol'], row['chap'], 'text-data/CC_ML_FR_trimmed_cleaned_EN_cl_MAlem')
#                 #print('non-am row')
#             else:
#                 #for full chapters, wordcount as usual and save into a temp variable for calculating fractions 
#                 if '.' not in row['chap']:
#                     row['word'] = wordcount_f(row['vol'], row['chap'], 'text-data/CC_ML_FR_trimmed_cleaned_EN_cl_MAlem')
#                     parent_wordcount = row['word'] + 0.0001 #avoid div-zero
#                     #print('am-row non-sectioned')
#             #for Am rows:
#             #check for sectioned text file in AMcat folder 
#             #when catches section: find corresponding full chapter;  + wordcount + sect_f = 1
#             # then write in all the sectioned rows while calculating fractions
#                 else:
#                     print('am sect check started')
#                     AM_filelist = os.scandir(cat_path)
#                     for entry in AM_filelist:
#                         # complicated slicing for chapter value as get_chap above is not tuned to segmented sections
#                         if int(get_vol(entry.name)) == int(row['vol']) and entry.name[entry.name.find('_')+1:entry.name.find('AMER')-1] == row['chap']:
#                                 print(row, 'am row sectioned chap check passed')
#                                 row['word'] = wordcount_f(row['vol'], row['chap'], 'text-data/CC_ML_FR_trimmed_MAlem_AMER_cat_EN')
#                                 row['sect_frac'] = row['word'] / parent_wordcount
#             #write row as composed in either decision tree branch
#             writer.writerow(row)

In [47]:
# entryname = '07_23.6_AMER_1497_A_note_of_Cabots_first_discoverie_from_Robert_Fabians_Chronicle_pp.154-154'
# print(entryname[entryname.find('_')+1:entryname.find('AMER')-1])

23.6


30-32.3
